# SQL exercises

PostgresSQL excercises from https://pgexercises.com/questions/basic/

![sql-exercise-schema](img/sql-exercise-schema.png)

- [Basic](#Basic)

Intermediate
- [Join](#Join)
- [Aggregation](#Aggregation)
- [Case statement](#Case)

Advance 
- [Cast](#Cast)
- [Rank](#Rank)
- [Window Function](#WINDOW-FUNCTION)
- [Timestamp](#Timestamp)

## Additional Practice
[Leetcode](#Leetcode)

# Basic

### Multiple conditions
How can you produce a list of facilities that charge a fee to members, and that fee is less than 1/50th of the monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

    SELECT facid, name, membercost, monthlymaintenance
        FROM cd.facilities
        WHERE (membercost > 0) AND (membercost < monthlymaintenance / 50)



### IN Function
How can you retrieve the details of facilities with ID 1 and 5? Try to do it without using the OR operator.

    SELECT *
        FROM cd.facilities
        WHERE facid IN (1,5)



### Datetime
How can you produce a list of members who joined after the start of September 2012? Return the memid, surname, firstname, and joindate of the members in question.


    SELECT memid, surname, firstname, joindate
        FROM cd.members
        WHERE joindate >= '2012-09-01'

### Union
You, for some reason, want a combined list of all surnames and all facility names. Yes, this is a contrived example :-). Produce that list!

    SELECT surname 
        FROM cd.members 
        UNION 
    SELECT name
        FROM cd.facilities

### Case

### CASE statement (CASE WHEN ... ELSE ... END AS ...)
How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive' depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.

     
     SELECT name,
        CASE WHEN monthlymaintenance > 100 THEN 
            'expensive'
        ELSE 
            'cheap'
        END AS 
            cost
        FROM cd.facilities

***

# Join

### Self join with aggregation
You'd like to get the first and last name of the last member(s) who signed up - not just the date. How can you do that?


    SELECT firstname, surname, joindate
        FROM cd.members
        WHERE joindate = (SELECT MAX(joindate) FROM cd.members)


**Explaination: Why this would not work**

    SELECT firstname, surname, MAX(joindate)
        FROM cd.members

MAX(joindate) - returns a single value when firstname and surname would return a list of all the name values, hence it was confused when asked to concate these three items together


**In the correct query, we have created one single row from the subquery and compare this row to all the rows in the joindate column.**


Other approach:


     SELECT firstname, surname, joindate
       FROM cd.members
       ORDER BY joindate DESC
       LIMIT 1


### Regex
How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.


    SELECT book.starttime as start, fac.name as name
        FROM cd.facilities as fac
        JOIN cd.bookings as book
        ON fac.facid = book.facid
        WHERE book.starttime::DATE = '2012-09-21' AND (fac.name ~ '^Tennis')
    ORDER BY book.starttime


### Self join (Distinct)
How can you output a list of all members who have recommended another member? Ensure that there are no duplicates in the list, and that results are ordered by (surname, firstname).

    SELECT firstname , surname
        FROM cd.members
        WHERE memid in (SELECT DISTINCT(recommendedby) FROM cd.members)
    ORDER BY 2,1
    
    
**Explaination**: 

SELECT DISTINCT(recommendedby) returns a list of distinct recommenders and match these ids to memid(member/recommender's actual id)

### Self join 
How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).



    SELECT A.firstname, A.surname, recommenders.firstname as recfname, 
        recommenders.surname as recsname
        FROM cd.members AS A
            LEFT JOIN (SELECT firstname, surname, memid
                       FROM cd.members) AS recommenders
            ON A.recommendedby = recommenders.memid
    ORDER BY A.surname, A.firstname

### (need revision)
How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user is always ID 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.


    SELECT firstname ||' ' || surname as member, 
    fac.name as facility, 
    CASE

    WHEN mem.memid != 0 THEN

        bk.slots * fac.membercost 

    ELSE	
        bk.slots * fac.guestcost

    END AS cost



    FROM cd.bookings bk

    JOIN cd.facilities fac

    ON bk.facid = fac.facid

    JOIN cd.members mem

    ON mem.memid = bk.memid

    WHERE starttime::date = '2012-09-14'

    AND (
      (bk.memid != 0 AND bk.slots * fac.membercost > 30 ) OR 
      (bk.memid = 0 AND bk.slots * fac.guestcost >30 ))


    ORDER BY cost DESC


***

# Aggregation

### Not Null

Produce a count of the number of recommendations each member has made. Order by member ID.

    SELECT recommendedby, COUNT(*)
        FROM cd.members
        WHERE recommendedby IS NOT NULL
        GROUP BY recommendedby
    ORDER BY recommendedby

### Filter on date and sum
Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.

    SELECT facid, SUM(slots) as "Total Slots"
        FROM cd.bookings bk
        WHERE date_trunc('month', starttime) >= '2012-09-01'
           AND date_trunc('month', starttime) <= '2012-09-30'
        GROUP BY facid
    ORDER BY 2


## Cast

### CAST Function
Produce a list of the total number of hours booked per facility, remembering that a slot lasts half an hour. The output table should consist of the facility id, name, and hours booked, sorted by facility id. Try formatting the hours to two decimal places.

    SELECT fac.facid, fac.name, CAST(SUM(slots)/2.0 AS DECIMAL(10,2)) AS "Total Hours"
        FROM cd.facilities fac
        JOIN cd.bookings bk
        ON fac.facid = bk.facid
        GROUP BY fac.facid
    ORDER BY 1,2

### Rollup Function
Produce a list of the total number of slots booked per facility per month in the year of 2012. In this version, include output rows containing totals for all months per facility, and a total for all months for all facilities. The output table should consist of facility id, month and slots, sorted by the id and month. When calculating the aggregated values for all months and all facids, return null values in the month and facid columns.

    SELECT facid, EXTRACT ('MONTH' FROM starttime) as "month" , SUM(slots) as "slots"
        FROM cd.bookings   
        WHERE EXTRACT ('YEAR' FROM starttime) = '2012'
        GROUP BY ROLLUP(facid, 2)
    ORDER BY 1, 2

## Rank

### RANK Function and ROUND Function
Produce a list of members, along with the number of hours they've booked in facilities, rounded to the nearest ten hours. Rank them by this rounded figure, producing output of first name, surname, rounded hours, rank. Sort by rank, surname, and first name.

    SELECT firstname, surname, CAST(ROUND(SUM(slots)/2, -1) AS DECIMAL(10,0)) as hours,
        RANK() OVER (ORDER BY CAST(ROUND(SUM(slots)/2, -1) AS DECIMAL(10,0)) DESC) AS rank

        FROM cd.members mem
        JOIN cd.bookings bk
        ON mem.memid = bk.memid
        GROUP BY mem.memid

    ORDER BY 4,2,1

## WINDOW FUNCTION


return a **single value for each row** from the underlying query.

### OVER() Function
Produce a list of member names, with each row containing the total member count. Order by join date.

    SELECT COUNT(*) OVER(), firstname, surname
        FROM cd.members
        ORDER BY joindate

### ROW_NUMBER FUNCTION

Produce a monotonically increasing numbered list of members, ordered by their date of joining. Remember that member IDs are not guaranteed to be sequential.

    SELECT ROW_NUMBER() OVER(ORDER BY joindate), firstname, surname
        FROM cd.members

***

# Timestamp

### Interval
Find the result of subtracting the timestamp '2012-07-30 01:00:00' from the timestamp '2012-08-31 01:00:00'

    SELECT timestamp '2012-08-31 01:00:00' - timestamp '2012-07-30 01:00:00' as interval

### generate_series Function


    SELECT generate_series(timestamp '2012-10-01', timestamp '2012-10-31', interval '1 day') as ts;          


### extract Function
Get the day of the month from the timestamp '2012-08-31' as an integer.

    SELECT EXTRACT (DAY FROM timestamp '2012-08-31')

### date_trunc Function

Return a count of bookings for each month, sorted by month

    SELECT  date_trunc ('month', starttime) as month, COUNT(*)
        FROM cd.bookings
        GROUP BY month
    ORDER BY month

***

# Leetcode

#### 176. Second Highest Salary

    SELECT  MAX(Salary) SecondHighestSalary
        FROM Employee
        WHERE Salary NOT IN (SELECT MAX(Salary) FROM Employee)
    

#### 182. Duplicate Emails

    SELECT Email
       FROM Person
       GROUP BY Email
       HAVING COUNT(*) > 1

#### 181. Employees Earning More Than Their Managers

    SELECT Employee.Name as Employee
        FROM Employee
        JOIN (SELECT Id, Salary FROM Employee) as man
        ON man.Id = Employee.ManagerId
        WHERE man.Salary <  Employee.Salary

#### 197. Rising Temperature

    SELECT W2.ID
        FROM Weather as W1, Weather as W2
        WHERE DATEDIFF(W2.RecordDate, W1.RecordDate) = 1 AND W2.Temperature > W1.Temperature 

#### 183. Customers Who Never Order

    SELECT Customers.Name as Customers
        FROM Customers 
        LEFT JOIN Orders
        ON Customers.Id = Orders.CustomerId
        WHERE Orders.CustomerId IS NULL